In [ ]:
!pip install pandas
!pip install numpy
!pip install faiss-cpu
!pip install sentence-transformers
!pip install langid

In [3]:
import glob
import pandas as pd
import numpy as np
import langid
import faiss
from sentence_transformers import SentenceTransformer

data_path = "./data"

2021-11-17 17:12:19.397866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-17 17:12:19.397898: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Introdução

### Busca Semantica

A busca semântica busca melhorar a precisão dos resultados, entendendo o conteúdo da consulta na pesquisa. Em contraste com os mecanismos de pesquisa tradicionais, que só encontram documentos com base em correspondências lexicais, a pesquisa semântica também pode encontrar sinônimos.

### Como é feito?

A ideia por trás da pesquisa semântica é incorporar todas as entradas do corpus (a.k.a. coleção de dados), sejam sentenças, parágrafos ou documentos, em um espaço vetorial. No momento da pesquisa, a consulta é inserida no mesmo espaço vetorial e os embeddings mais próximos do seu corpus são encontrados. Essas entradas devem ter uma alta sobreposição semântica com a consulta.

![img01](./img/SemanticSearch.png)

### O que é Word Embedding?

Inicialmente, é preciso transformar o texto em uma informação numérica, mais especificamente um vetor. Existem algumas formas de se fazer isso, uma delas é definir cada palavra através de escalas representando alguma informação sobre essa palavra.

![img02](./img/emb01.png)
![img03](./img/emb02.png)

# SentenceTransformers

SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in our paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks.

In [10]:
%%time
model = SentenceTransformer('all-mpnet-base-v2')

CPU times: user 1.88 s, sys: 649 ms, total: 2.53 s
Wall time: 12.3 s


# Dataframe

In [3]:
data_files = glob.glob(f'{data_path}/*.pkl')
li = []

for filename in data_files:
    df = pd.read_pickle(filename)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt13939432,tvEpisode,Episode #1.1,Episode #1.1,0,2021,\N,\N,"Music,Talk-Show"
1,tt13939434,tvEpisode,Ep 31,Ep 31,0,1997,\N,\N,"Drama,History"
2,tt13939436,tvEpisode,Knit It to Win It,Knit It to Win It,0,2021,\N,\N,Reality-TV
3,tt13939438,tvEpisode,Episode #1.2,Episode #1.2,0,2021,\N,\N,"Music,Talk-Show"
4,tt1393943,tvEpisode,RFC II: Revenge of the Warriors,RFC II: Revenge of the Warriors,0,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...
8468610,tt10158948,tvEpisode,Episode #1.228,Episode #1.228,0,2011,\N,\N,Drama
8468611,tt1015894,tvEpisode,Episode #1.6,Episode #1.6,0,2003,\N,\N,Documentary
8468612,tt10158950,short,My Father My King,My Father My King,0,2019,\N,15,"Drama,Short"
8468613,tt10158952,tvEpisode,Episode #1.229,Episode #1.229,0,2011,\N,\N,Drama


# Pre Processamento

In [4]:
df.drop(['isAdult','endYear'], axis=1, inplace=True)
df.replace(r'\N', np.NaN, inplace=True)
df.dropna(subset=['startYear'], inplace=True)
df.dropna(subset=['genres'], inplace=True)
df['startYear'] = df['startYear'].astype(int)

df.drop(df.loc[df['titleType'] != 'movie'].index, inplace=True)
df.drop(df.loc[df['startYear'] < 2010].index, inplace=True)
df.drop(df.loc[df['genres'].str.contains("Documentary")].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt13939540,movie,Nandeesha,Nandeesha,2012,NaN,Drama
1,tt13939676,movie,Sri Chowdeshwari Devi Mahime,Sri Chowdeshwari Devi Mahime,2012,NaN,Drama
2,tt13939738,movie,Hosa Prema Purana,Hosa Prema Purana,2012,NaN,Drama
3,tt13939774,movie,See You,See You,2012,NaN,Drama
4,tt13939944,movie,Kamsale Kaisale,Kamsale Kaisale,2012,90,Drama
...,...,...,...,...,...,...,...
124195,tt10157796,movie,Menka Urvashi,Menka Urvashi,2019,149,Drama
124196,tt10157810,movie,Fjols til Fjells,Fjols til Fjells,2020,87,Comedy
124197,tt10157842,movie,Single 2,Single 2,2019,128,"Comedy,Romance"
124198,tt10158628,movie,Tokan,Tokan,2013,83,"Action,Crime"


 ## Detectando a língua do título

 Removendo títulos que não estão em inglês

In [5]:
def detect_language(doc):
    lang, log_prob = langid.classify(doc)
    if lang != 'en':
        return np.nan
    return doc

df['primaryTitle'] = df['primaryTitle'].apply(detect_language)
df.dropna(subset=['primaryTitle'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt13939774,movie,See You,See You,2012,NaN,Drama
1,tt13940578,movie,Gokula Krishna,Gokula Krishna,2012,NaN,Drama
2,tt13941102,movie,The Corsicana Eight,The Corsicana Eight,2021,NaN,Comedy
3,tt1394211,movie,Helena from the Wedding,Helena from the Wedding,2010,89,"Comedy,Drama"
4,tt1394240,movie,No Limit Kids: Much Ado About Middle School,No Limit Kids: Much Ado About Middle School,2010,91,"Comedy,Family"
...,...,...,...,...,...,...,...
86020,tt10157748,movie,Ave Rara,Ave Rara,2019,45,Drama
86021,tt10157796,movie,Menka Urvashi,Menka Urvashi,2019,149,Drama
86022,tt10157842,movie,Single 2,Single 2,2019,128,"Comedy,Romance"
86023,tt10158628,movie,Tokan,Tokan,2013,83,"Action,Crime"


## Salvando o dataframe pré-processado

In [6]:
df.to_pickle(f'{data_path}/data_pre.pkl')

# Faiss (Gerando o index)

Facebook AI Similarity Search (Faiss), permite uma busca eficiente por similaridade. Dado um conjunto de vetores, podemos indexá-los usando Faiss - então usando outro vetor (o vetor de consulta ), procuramos os vetores mais semelhantes dentro do índice. O Faiss não apenas permite construir um índice e pesquisar, mas também acelera tempos de pesquisa a níveis melhores de desempenho por neio do uso da GPU.

![img02](./img/similarity-search-indexes3.png)
![img03](./img/similarity-search-indexes4.png)

- IndexFlatL2 usa a distancia Euclediana;
- IndexFlatIP utiliza o produto da distancia.

In [13]:
df = pd.read_pickle(f'{data_path}/data_pre.pkl')

In [14]:
%%time
sentences = [df.originalTitle[x] for x in range(df.shape[0])]
sentence_embeddings = model.encode(sentences)
sentence_embeddings = np.asarray(sentence_embeddings.astype('float32'))

CPU times: user 10min 20s, sys: 4.84 s, total: 10min 25s
Wall time: 10min 25s


In [17]:
sentence_embeddings.shape

(86025, 768)

In [8]:
def generate_index():
    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
    ids = np.array(range(0, len(data)))
    ids = np.asarray(ids.astype('int64'))
    index.add_with_ids(sentence_embeddings, ids)
    faiss.write_index(index, f'{data_path}/sentences.index')

generate_index()

# Busca

In [14]:
def search(query):
    index = faiss.read_index(f'{data_path}/sentences.index')
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    results = top_k[1].tolist()[0]
    return df.iloc[results]

In [15]:
#query = "Space Fights with no sound"
query = input()
search(query)

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
5053,tt15156420,movie,Space,Space,2016,NaN,Mystery
13362,tt1858536,movie,Space,Space,2011,82,"Comedy,Drama"
31272,tt8710832,movie,Space,Space,2020,93,"Horror,Sci-Fi"
26305,tt7673996,movie,The Space,The Space,2018,70,Drama
79005,tt10613176,movie,Space Baby,Space Baby,2021,NaN,"Family,Fantasy,Sci-Fi"


# Validação

In [4]:
index = faiss.read_index(f'{data_path}/sentences.index')
index

<faiss.swigfaiss.IndexIDMap; proxy of <Swig Object of type 'faiss::IndexIDMapTemplate< faiss::Index > *' at 0x7f7294578de0> >

In [8]:
vecs = np.zeros((5, 768))

for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

NameError: name 'I' is not defined